# Configuration basics

Ablator has the ability to dynamically create a hierarchical configuration by composition and override it through config files and the command line.

Important configuration classes are presented below. Note that you can either store all these parameters in a yaml config file, or you can play around with classes. You can refer to [these examples]() to see how these 2 cases are done in practice.

## Configuration parameters

For our framework, configuration is organized into different categories: running configs (either for training a single model or training multiple models in parallel), model configs, training configs, optimizer configs, scheduler configs. Most of these will be used together in order for `ablator` to work seamlessly.

### RunConfig

`RunConfig` is used for setting up configuration for the experiment to run, for example, where to store experiment artifacts, the device to be used (gpu, cpu), when to do validation step, when to do progress logging. 

The table below summarizes parameters that one can use and need to define. Note that `RunConfig` requires TrainConfig and ModelConfig to be included during initialization. Learn more about these in the next sections of this tutorial.

| Parameter           | Usage                                                                                                                                                                                                                                                                                                             |
|---------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| experiment_dir      | location to store experiment artifacts.                                                                                                                                                                                                                                                                           |
| random_seed         | random seed.                                                                                                                                                                                                                                                                                                      |
| train_config        | training configuration. (check ``TrainConfig`` for more details)                                                                                                                                                                                                                                                  |
| model_config        | model configuration. (check ``ModelConfig`` for more details)                                                                                                                                                                                                                                                     |
| keep_n_checkpoints  | number of latest checkpoints to keep.                                                                                                                                                                                                                                                                             |
| tensorboard         | whether to use tensorboardLogger.                                                                                                                                                                                                                                                                                 |
| amp                 | whether to use automatic mixed precision when running on gpu.                                                                                                                                                                                                                                                     |
| device              | device to run on.                                                                                                                                                                                                                                                                                                 |
| verbose             | verbosity level.                                                                                                                                                                                                                                                                                                  |
| eval_subsample      | fraction of the dataset to use for evaluation.                                                                                                                                                                                                                                                                    |
| metrics_n_batches   | max number of batches stored in every tag(train, eval, test) for evaluation.                                                                                                                                                                                                                                      |
| metrics_mb_limit    | max number of megabytes stored in every tag(train, eval, test) for evaluation.                                                                                                                                                                                                                                    |
| early_stopping_iter | The maximum allowed difference between the current iteration and the last <br />iteration with the best metric before applying early stopping. Early stopping <br />will be triggered if the difference ``(current_itr-best_itr)`` exceeds ``early_stopping_iter``.<br />If set to ``None``, early stopping will not be applied. |
| eval_epoch          | The epoch interval between two evaluations.                                                                                                                                                                                                                                                                       |
| log_epoch           | The epoch interval between two logging.                                                                                                                                                                                                                                                                           |
| init_chkpt          | path to a checkpoint to initialize the model with.                                                                                                                                                                                                                                                                |
| warm_up_epochs      | number of epochs marked as warm up epochs.                                                                                                                                                                                                                                                                        |
| divergence_factor   | if ``cur_loss > best_loss > divergence_factor``, the model is considered <br />to have diverged.                                                                                                                                                                                                                        |

### ParallelConfig

`ParallelConfig` is a subclass of `RunConfig`. It extends the functionality by introducing additional arguments to configure parallel training, enabling horizontal scaling of a single experiment. It helps define the setting of distributed training in the experiment, such as specifying the number of trials, the maximum number of trials to run concurrently, the target metrics to optimize, and more.

| Parameter             | Usage                                                                                                                                              |
|-----------------------|----------------------------------------------------------------------------------------------------------------------------------------------------|
| total_trials          | total number of trials.                                                                                                                            |
| concurrent_trials     | number of trials to run concurrently.                                                                                                              |
| search_space          | search space for hyperparameter search,eg. ``{"train_config.optimizer_config.arguments.lr": SearchSpace(value_range=[0, 10], value_type="int"),}`` |
| optim_metrics         | metrics to optimize, eg. ``{"val_loss": "min"}``                                                                                                   |
| search_algo           | type of search algorithm.                                                                                                                          |
| ignore_invalid_params | whether to ignore invalid parameters when sampling.                                                                                                |
| remote_config         | remote storage configuration.                                                                                                                      |
| gcp_config            | gcp configuration.                                                                                                                                 |
| gpu_mb_per_experiment | gpu resource to assign to an experiment.                                                                                                           |
| cpus_per_experiment   | cpu resource to assign to an experiment.                                                                                                           |

It's worth mentioning `search_space`, which is used to define a set of categorical or discrete/continuous values for a certain hyperparameter that you want to ablate. Refer to [this]() to learn more about how to use it for ablation.

### ModelConfig

This configuration can be used to add parameters specific to the model you're using. A sample use case for this is when you want to try different model sizes, number of layers, activation functions, etc. By creating `ModelConfig` class for your model and include these parameters, `ablator` will be able to create search space over them, hence able to run Hyperparameter optimization.

There are 2 extra steps you need to do if you create a custom config from `ModelConfig`. Firstly, since you have defined model parameters to be customized, when creating the model module, you have to pass to its constructor this custom config, then construct the model using this its parameters. Secondly, as the running config (`RunConfig` or `ParallelConfig`) has `model_config` attribute, you will need to create a custom running config class as well (decorated with `configclass` decorator), updating `model_config` argument to proper type. Only after this can the running config object be used in `ablator` launcher.

Note that in the model config class, all arguments should be defined as Stateless or Derived data type. These are custom Python annotations to define attributes to which the experiment state is agnostic.

- Stateless configuration attributes can be used as a proxy for variables that can take different value assignments between trials or experiments. For example, the learning rate can be set as an independent variable and must be annotated as stateless. Additionally, there are variables that take different values between experiments and trials to which the state is agnostic, for example, a random seed or a directory path between execution environments can be annotated as stateless.

- Derived attributes are un-decided at the start of the experiment and do not require a value assignment. Instead, the value is determined by internal experiment processes that can depend on other experimental attributes, such as the dataset. However, given the same initial state, the attribute is expected to result in the same value and is therefore deterministic. For example, the input size used in a model’s architecture that depends on the dataset will be annotated as Derived during the experiment design phase.

Here's a concrete example of a simple 1-layer neural network model, with injected configuration for input size, hidden layer dimension, activation function, and dropout rate.

In [6]:
from ablator import RunConfig, ModelConfig, Stateless, Derived, configclass

import torch.nn as nn
import torch

class MyModelConfig(ModelConfig):
    inp_size: Derived[int]
    hidden_dim: Stateless[int]
    activation: Stateless[str]
    dropout: Stateless[float]

@configclass
class CustomRunConfig(RunConfig):
    model_config: MyModelConfig

class MyCustomModel(nn.Module):
    def __init__(self, config: MyModelConfig) -> None:
        super().__init__()
        self.linear = nn.Linear(config.inp_size, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)
        if config.activation == "relu":
            self.activate = nn.ReLU()
        elif config.activation == "elu":
            self.activate = nn.ELU()

    def forward(self, x: torch.Tensor):
        out = self.linear(x)
        out = self.dropout(out)
        out = self.activate(out)

        return {"preds": out, "labels": out}, x.sum().abs()

### TrainConfig

This configuration class defines everything that is related to the main training process of your model, which includes dataset name, batch size, number of epochs, optimizer, scheduler. 2 important attributes to metion are `optimizer_config` and `scheduler_config`. As the names suggest, they configure the optimizer and scheduler to be used in the training process.

| Parameter         | Usage                                                                 |
|-------------------|-----------------------------------------------------------------------|
| dataset           | dataset name. maybe used in custom dataset loader functions.          |
| batch_size        | batch size.                                                           |
| epochs            | number of epochs to train.                                            |
| optimizer_config  | optimizer configuration. (check ``OptimizerConfig`` for more details) |
| scheduler_config  | scheduler configuration. (check ``SchedulerConfig`` for more details) |
| rand_weights_init | whether to initialize model weights randomly.                         |

### OptimizerConfig and SchedulerConfig

`OptimizerConfig` is a config class that allows user choose the optimizer they wanted. Currently, we supports SGD optimizer, Adam optimizer, and AdamW optimizer.

`SchedulerConfig`, on the other hand, can be used for scheduling learning rate updates in the training process.

Both of these config classes have similar arguments:

| Parameter | Usage                                                                                                                                                                       |
|-----------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| name      | The type of the scheduler or optimizer, this can be any in ``['None', 'step', 'cycle', 'plateau']``  for schedulers and in ``['sgd', 'adam', 'adamw']`` for optimizers. |
| arguments | The arguments for the scheduler or optimizer, specific to a certain type of scheduler or scheduler.                                                                         |

The table below shows how arguments can be defined for each type of scheduler:

| Scheduler type | Arguments                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|----------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| cycle          | `max_lr` : Upper learning rate boundaries in the cycle.  `total_steps` : The total number of steps to run the scheduler in a cycle.  `step_when` : The step type at which the `scheduler.step()` should be invoked: ``'train'``, ``'val'``, or ``'epoch'``.             |
| plataeu        | `patience` : Number of epochs with no improvement after which learning rate will be reduced.  `min_lr` : A lower bound on the learning rate. `mode` : One of ``'min'``, ``'max'``, or ``'auto'``, which defines the  direction of optimization, so as to adjust the learning rate accordingly, i.e when a certain metric ceases improving.  `factor` : Factor by which the learning rate will be reduced. ``new_lr = lr * factor``.  `threshold` : Threshold for measuring the new optimum, to only focus on significant changes.  `verbose` : If ``True``, prints a message to ``stdout`` for each update.  `step_when` : The step type at which the scheduler should be invoked: ``'train'``, ``'val'``, or ``'epoch'``.  |
| step           | `step_size` : Period of learning rate decay, by default 1.  `gamma` : Multiplicative factor of learning rate decay, by default 0.99.  `step_when` : The step type at which the scheduler should be invoked: ``'train'``, ``'val'``, or ``'epoch'``.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |

The table below shows how arguments can be defined for each type of optimzer:

| Optimizer type | Arguments                                                                                                                                                                                                                                                                           |
|----------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| sgd            | `weight_decay` : Weight decay rate<br> `momentum` : Momentum factor<br>                                                                                                                                                                                                           |
| adam           | `betas` : Coefficients for computing running averages of gradient and its square (default is ``(0.5, 0.9)``).<br/> `weight_decay` : Weight decay rate (default is ``0.0``).<br/>                                                                                                    |
| adamw          | `betas` : Coefficients for computing running averages of gradient and its square (default is ``(0.9, 0.999)``).<br/> `eps` : Term added to the denominator to improve numerical stability (default is ``1e-8``).<br/> `weight_decay` : Weight decay rate (default is ``0.0``).<br/> |

The following code snippet describes how to initialize these objects

In [1]:
from ablator import OptimizerConfig, SchedulerConfig

optimizer_config = OptimizerConfig(name="sgd", arguments={"lr": 0.1})
scheduler_config = SchedulerConfig(name="cycle", arguments={"max_lr": 0.5, "total_steps": 50})

Now let's combine everything. Ablator trainer requires a model wrapper and a running config when initializing, after that, experiment can be launched via `trainer.launch()`. This tutorial focuses on helping you define the running configuration `run_config`.

Take the code snippet below as an example, `train_config` sets up the dataset, batch size, epochs, and references the optimizer configuration. Next, `config` object combines the `training_config` and `model_config`, along with runtime settings like verbosity and device.

In [ ]:
from ablator import TrainConfig

train_config = TrainConfig(
    dataset="test",
    batch_size=128,
    epochs=2,
    optimizer_config=optimizer_config,
    scheduler_config=scheduler_config,
)

config = CustomRunConfig(
    train_config=train_config,
    model_config=MyModelConfig(),
    verbose="silent",
    device="cpu",
)

With the configuration created, we are half-way to running ablation experiment with the ablator trainer.
```
trainer = ParallelTrainer(wrapper=model_wrapper, run_config=run_config)
trainer.launch()
```
In the next chapter, you will learn how to create the model wrapper, the other half that's left. We will start with training a single model.

## Different methods to define running configurations

There are 3 ways to provide values to the configurations: named arguments, file-based, or dictionary-based. All examples from the previous sections are actually the named arguments method. Now let's look at how file based method and dictionary based method work.

### File-based

File based configuration is a way for you to create simple configuration files, passing configuration values to a single `yaml` file. After that, based on the type of running configuration you want, you can use `RunConfigClass.load(path/to/yaml/file)` method to create configuration with values provided in the config file.

To write these config files, simply follow `key : value` syntax (each pair on a single line). The following example shows what a config yaml file looks like. We will name it `config.yaml`:
```
experiment_dir: "/tmp/dir"
train_config:
  dataset: cifar10
  optimizer_config:
    name: adam
    arguments:
      lr: 0.01
      weight_decay: 0.0
  batch_size: 128
  epochs: 10
  scheduler_config: null
  rand_weights_init: true
model_config:
  inp_size: 50
  hidden_dim: 100
  activation: "relu"
  dropout: 0.15
metrics_n_batches: 400
```
We can see that the outermost arguments are from `RunConfig`. Also note how `train_config`, which corresponds to `TrainConfig` object in the running config, has its arguments defined 1 level below (indented). Therefore, the first rule to follow is that the arguments to use are from the running config class, either `RunConfig` or `ParallelConfig`, so make sure you use the right set of arguments. The second rule is that any arguments that is another config class should be indented 1 level from its parent config class.

Now in your code, only 1 single line of code is required to load these values to create the config object:
```
config = CustomRunConfig.load("path/to/yaml/file")
```

Note that since we created a custom running configuration class `CustomRunConfig` that is tied to the custom model config in the previous sections, we used `CustomRunConfig.load("path/to/yaml/file")` to load configuration from file. Otherwise, if you're not creating any subclasses, `RunConfig.load("path")` or `ParallelConfig.load("path")` also works.

### Dictionary based

Create dictionary